In [1]:
# modeling

import csv
import cv2
import numpy as np

lines = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
for line in lines:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = './data/IMG/'+filename
        image = cv2.imread(current_path)
        images.append(image)
        if i == 1:
            measurement = float(line[3]) + 0.2
        elif i == 2:
            measurement = float(line[3]) - 0.2
        else:
            measurement = float(line[3])      
        measurements.append(measurement)
        
#         images.append(cv2.flip(image,1))
#         measurements.append(measurement*-1.0)
    
print(len(images))
print(len(measurements))
        
X_train = np.array(images) 
y_train = np.array(measurements)

7899
7899


In [6]:
import os
import csv

samples = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

import cv2
import numpy as np
import sklearn
from sklearn.utils import shuffle

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                for i in range(3):
                    name = './data/IMG/'+batch_sample[i].split('/')[-1]
                    image = cv2.imread(name)
                    images.append(image)
                    if i == 1:
                        angle = float(batch_sample[3]) + 0.2
                    elif i == 2:
                        angle = float(batch_sample[3]) - 0.2
                    else:
                        angle = float(batch_sample[3])    
                    angles.append(angle)

                    images.append(cv2.flip(image,1))
                    angles.append(angle*-1.0)                

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D


model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24, 5, 5, subsample=(2,2),activation="relu"))
model.add(Convolution2D(36, 5, 5, subsample=(2,2),activation="relu"))
model.add(Convolution2D(48, 5, 5, subsample=(2,2),activation="relu"))
model.add(Convolution2D(64, 3, 3, activation="relu"))
model.add(Convolution2D(64, 3, 3, activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch= len(train_samples), 
                    validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=2)

model.save('model_2.h5')

C:\Users\10001161181\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:62: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="relu")`
C:\Users\10001161181\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="relu")`
C:\Users\10001161181\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:64: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2), activation="relu")`
C:\Users\10001161181\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:65: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
C:\Users\10001161181\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykern

Epoch 1/2
2106/2106 [==============================] - 4934s - loss: 0.0114 - val_loss: 0.0240
Epoch 2/2
2106/2106 [==============================] - 4927s - loss: 0.0032 - val_loss: 0.0238


In [2]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))

model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24, 5, 5, subsample=(2,2),activation="relu"))
model.add(Convolution2D(36, 5, 5, subsample=(2,2),activation="relu"))
model.add(Convolution2D(48, 5, 5, subsample=(2,2),activation="relu"))
model.add(Convolution2D(64, 3, 3, activation="relu"))
model.add(Convolution2D(64, 3, 3, activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=2)

model.save('model_2.h5')

Using TensorFlow backend.
C:\Users\10001161181\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\10001161181\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\10001161181\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  if sys.path[0] == '':
C:\Users\10001161181\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D`

Train on 6319 samples, validate on 1580 samples
Epoch 1/2
6319/6319 [==============================] - 80s - loss: 0.0241 - val_loss: 0.0439
Epoch 2/2
6319/6319 [==============================] - 80s - loss: 0.0192 - val_loss: 0.0537


In [1]:
import argparse
import base64
from datetime import datetime
import os
import shutil

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO

from keras.models import load_model
import h5py
from keras import __version__ as keras_version
from keras.layers import Cropping2D

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None


class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error

        return self.Kp * self.error + self.Ki * self.integral


controller = SimplePIController(0.1, 0.002)
set_speed = 9
controller.set_desired(set_speed)


@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = data["steering_angle"]
        # The current throttle of the car
        throttle = data["throttle"]
        # The current speed of the car
        speed = data["speed"]
        # The current image from the center camera of the car
        imgString = data["image"]
        image = Image.open(BytesIO(base64.b64decode(imgString)))
        image_array = np.asarray(image)
        steering_angle = float(model.predict(image_array[None, :, :, :], batch_size=1))

        throttle = controller.update(float(speed))

        print(steering_angle, throttle)
        send_control(steering_angle, throttle)

        # save frame
#         if args.image_folder != '':
        timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
        image_filename = os.path.join('./movie', timestamp)
        image.save('{}.jpg'.format(image_filename))
    else:
        # NOTE: DON'T EDIT THIS.
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)

Using TensorFlow backend.


In [2]:
# from moviepy.editor import ImageSequenceClip
# import argparse

if __name__ == '__main__':
    model = load_model('model_2.h5')
    print(model.summary())

#     if args.image_folder != '':
#         print("Creating image folder at {}".format(args.image_folder))
#         if not os.path.exists(args.image_folder):
#             os.makedirs(args.image_folder)
#         else:
#             shutil.rmtree(args.image_folder)
#             os.makedirs(args.image_folder)
#         print("RECORDING THIS RUN ...")
#     else:
#     print("NOT RECORDING THIS RUN ...")

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

    
#     video_file = 'video.mp4'
#     clip = ImageSequenceClip('./', fps=48)
#     clip.write_videofile(video_file)

C:\Users\10001161181\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:281: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
(3252) wsgi starting up on http://0.0.0.0:4567


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_3 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 31, 158, 24)       1824      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 77, 36)        21636     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 5, 37, 48)         43248     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 3, 35, 64)         27712     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 1, 33, 64)         36928     
__________

(3252) accepted ('127.0.0.1', 62684)


connect  9bf12224140e4970ae0babbbd26beed0
-0.27176040410995483 0.7392960000000001
-0.2837035655975342 0.66444
-0.2837035655975342 0.677184
-0.3159942030906677 0.2431782
-0.3159942030906677 0.2471624
-0.3159942030906677 0.2511466
-0.012956637889146805 0.5958720000000001
-0.012956637889146805 0.6065374
-0.012956637889146805 0.6172028
-0.14881201088428497 0.9253614000000001
-0.14881201088428497 0.94186
-0.14881201088428497 0.9583586000000001
-0.09465638548135757 0.92298
-0.09465638548135757 0.9384614000000001
-0.09465638548135757 0.9539428000000001
0.008456223644316196 0.9071838
0.008456223644316196 0.9214448
0.008456223644316196 0.9357058
-0.1259957104921341 0.8578302000000001
-0.1259957104921341 0.8702846000000001
-0.1259957104921341 0.882739
-0.09797921031713486 0.7780566
-0.09797921031713486 0.7882142000000001
-0.09797921031713486 0.7983718
0.1364428997039795 0.6786120000000001
0.1364428997039795 0.6862222000000001
0.1364428997039795 0.6938324000000001
-0.13441739976406097 0.571780200

0.06852998584508896 0.08069279999999991
0.06852998584508896 0.0806643999999999
0.06852998584508896 0.0806359999999999
0.11502809822559357 0.08108700000000003
0.11502809822559357 0.08106800000000003
0.11502809822559357 0.08104900000000002
-0.0062276218086481094 0.08122379999999994
-0.0062276218086481094 0.08120859999999995
-0.0062276218086481094 0.08119339999999996
-0.050164759159088135 0.08090279999999987
-0.050164759159088135 0.08088219999999988
-0.050164759159088135 0.08086159999999987
-0.021947793662548065 0.08041259999999997
-0.021947793662548065 0.08038359999999997
-0.021947793662548065 0.08035459999999997
0.08403287082910538 0.07987679999999993
0.08403287082910538 0.07983899999999992
0.08403287082910538 0.07980119999999992
-0.1271085888147354 0.07898819999999991
-0.1271085888147354 0.07893519999999991
-0.1271085888147354 0.07888219999999992
-0.1374758630990982 0.07880879999999997
-0.1374758630990982 0.07875539999999998
-0.1374758630990982 0.07870199999999998
-0.13419891893863678 

0.06523223221302032 0.07851819999999983
0.022750746458768845 0.08010959999999989
0.022750746458768845 0.0801309999999999
0.022750746458768845 0.0801523999999999
0.18186363577842712 0.08212199999999989
0.18186363577842712 0.0821815999999999
0.18186363577842712 0.0822411999999999
0.013709470629692078 0.08169899999999984
0.013709470629692078 0.08174679999999984
0.013709470629692078 0.08179459999999984
-0.183273047208786 0.08262779999999981
-0.183273047208786 0.0826909999999998
-0.183273047208786 0.0827541999999998
-0.09322044998407364 0.08214419999999992
-0.09322044998407364 0.08219419999999991
-0.09322044998407364 0.0822441999999999
-0.14729703962802887 0.08380379999999983
-0.14729703962802887 0.08388339999999983
-0.14729703962802887 0.08396299999999983
0.09469553083181381 0.08299199999999993
0.09469553083181381 0.08305099999999992
0.09469553083181381 0.08310999999999992
0.02386781945824623 0.08211839999999983
0.02386781945824623 0.08215679999999982
0.02386781945824623 0.0821951999999998

-0.13890644907951355 0.07817959999999985
0.050014678388834 0.07823159999999989
0.050014678388834 0.07822359999999988
0.050014678388834 0.07821559999999989
0.14758329093456268 0.0805739999999998
0.14758329093456268 0.0806123999999998
0.14758329093456268 0.0806507999999998
-0.04726165533065796 0.08122979999999981
-0.04726165533065796 0.08127879999999982
-0.04726165533065796 0.08132779999999983
-0.109967902302742 0.08167259999999985
-0.109967902302742 0.08172739999999984
-0.109967902302742 0.08178219999999985
-0.13541503250598907 0.08207159999999984
-0.13541503250598907 0.08213099999999983
-0.13541503250598907 0.08219039999999983
-0.18803341686725616 0.08185199999999986
-0.18803341686725616 0.08190359999999985
-0.18803341686725616 0.08195519999999987
-0.06991056352853775 0.0813335999999998
-0.06991056352853775 0.08137199999999979
-0.06991056352853775 0.08141039999999979
-0.2023080438375473 0.0810917999999999
-0.2023080438375473 0.08112319999999991
-0.2023080438375473 0.08115459999999991
0

-0.0025733932852745056 0.07915419999999967
-0.0025733932852745056 0.07912419999999967
-0.0025733932852745056 0.07909419999999967
-0.10095202922821045 0.07919679999999972
-0.10095202922821045 0.07916939999999972
-0.10095202922821045 0.07914199999999973


127.0.0.1 - - [25/Aug/2017 12:55:32] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 37.790779
wsgi exiting
(3252) wsgi exited, is_accepting=True
